# Word Embeddings
    More details in the official documentation: https://radimrehurek.com/gensim/auto_examples/index.html#documentation

In [8]:
!pip install gensim --upgrade
!pip install numpy --upgrade

In [9]:
from gensim import downloader
import numpy as np
import pandas as pd
import sklearn
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score

In [10]:
import gensim
print(gensim.__version__)

4.1.2


## Loading The Pretrained Weights
Supported options are at https://radimrehurek.com/gensim/models/word2vec.html#pretrained-models

In [11]:
import matplotlib as plt
WORD_2_VEC_PATH = 'word2vec-google-news-300'
GLOVE_PATH = 'glove-twitter-200'
GLOVE_PATH2= 'glove-wiki-gigaword-300'

Stav uses Glove_path

Doron uses Glove_path2 - Was pretty bad


In [12]:
glove = downloader.load(GLOVE_PATH)
#glove2=downloader.load(GLOVE_PATH2)
#word2vec=donwnloader.load(WORD_2_VEC_PATH)

In [13]:

train_path = 'train.tagged'
with open(train_path, 'r', encoding='utf-8') as f:
    train_sentences = f.readlines()
train_sentences = [sen.strip().lower() for sen in train_sentences]
train_sentences = [sen.split() for sen in train_sentences if sen]

dev_path = 'dev.tagged'
with open(dev_path, 'r', encoding='utf-8') as f:
    dev_sentences = f.readlines()
dev_sentences = [sen.strip().lower() for sen in dev_sentences]
dev_sentences = [sen.split() for sen in dev_sentences if sen]



# Preprocessing the Data 

Let's start with preprocessing the train data

In [14]:
words=[]
labels=[]
for i,word in enumerate(train_sentences):
  words.append(word[0])
  if word[1]=="o":
      labels.append(False)
  else:
      labels.append(True)


In [15]:
x_train = []
y_train=[]
miss_words_train=[]
for loc,word in enumerate(words):
    if word not in glove.key_to_index:
        #print(f"{word} not an existing word in the model")
        miss_words_train.append([labels[loc]])
        x_train.append(np.zeros((200,)))
        y_train.append(False)
        
        continue
    vec = glove[word]
    x_train.append(vec)
    y_train.append(labels[loc])
x_train = np.asarray(x_train)
y_train=np.asarray(y_train)
print(x_train.shape,y_train.shape)
df=pd.DataFrame(miss_words_train)
df.value_counts()

(46469, 200) (46469,)


False    5586
True      176
dtype: int64

now let's do the same for the Dev Set 

In [16]:
words=[]
labels=[]
for i,word in enumerate(dev_sentences):
  words.append(word[0])
  if word[1]=="o":
      labels.append(False)
  else:
      labels.append(True)


In [17]:
x_val = []
y_val=[]
miss_words_dev=[]

for loc,word in enumerate(words):
    if word not in glove.key_to_index:
        ##print(f"{word} not an existing word in the model")
        x_val.append(np.zeros((200,)))
        y_val.append(False)
        miss_words_dev.append(labels[loc])
        continue
    vec = glove[word]
    x_val.append(vec)
    y_val.append(labels[loc])
x_val = np.asarray(x_val)
y_val=np.asarray(y_val)
print(x_val.shape,y_val.shape)
df=pd.DataFrame(miss_words_dev)
df.value_counts()

(16261, 200) (16261,)


False    3011
True      115
dtype: int64

# Models

**AdaBoost Model**

In [ ]:
clf = AdaBoostClassifier(n_estimators=1000,random_state=20) # 8 mins for 1000 est
clf.fit(x_train, y_train)
train_y_pred=clf.predict(x_train)
val_y_pred=clf.predict(x_val)

In [ ]:

print(f'F1 score for the train {f1_score(y_train, train_y_pred)}')
print(f'F1 score for the val {f1_score(y_val, val_y_pred)}')


F1 score for the train 0.7646026573075959
F1 score for the val 0.528658875904285


In [17]:
from datetime import datetime

def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [ ]:
"""from sklearn.model_selection import  GridSearchCV
search_grid={'n_estimators':[1200,1500,2000],
             'learning_rate':[0.5,0.8,1]}
search=GridSearchCV(estimator=AdaBoostClassifier(),param_grid=search_grid,scoring='f1',n_jobs=1,cv=2,verbose=3)
start_time = timer(None) # timing starts from this point for "start_time" variable
search.fit(x_train, y_train)
timer(start_time) # timing ends here for "start_time" variable
print(search.best_estimator_)
print(search.best_params_)
"""

# SVM

In [20]:
from sklearn import svm
from sklearn.metrics import accuracy_score
svm_model = svm.SVC(random_state=20,C=2)
svm_model.fit(x_train, y_train)
val_y_pred_svm=svm_model.predict(x_val)
train_y_pred_svm=svm_model.predict(x_train)

print(f'F1 score for the train {f1_score(y_train, train_y_pred_svm)}')
print(f'Accuracy score for the train {accuracy_score(y_train, train_y_pred_svm)}')

print(f'F1 score for the val {f1_score(y_val, val_y_pred_svm)}')
print(f'Accuracy score for the val {accuracy_score(y_val, val_y_pred_svm)}')



F1 score for the train 0.8147227057945784
Accuracy score for the train 0.9839678064946523
F1 score for the val 0.6288009179575446
Accuracy score for the val 0.9602115491052211


C=2

F1 score for the train 0.789580171977744

Accuracy score for the train 0.9820955906087929

F1 score for the val 0.6362038664323374

Accuracy score for the val 0.9618104667609618


**Grid Search for SVM**

In [ ]:
from sklearn.model_selection import  GridSearchCV


param_grid = {'C':[2,3,4,5,6,7,10,20],'gamma':['scale','auto'],  'kernel':['rbf']}
grid = GridSearchCV(svm.SVC(),param_grid,refit = True, verbose=3,scoring='f1', cv=4)
start_time = timer(None) # timing starts from this point for "start_time" variable

grid.fit(x_train,y_train)
timer(start_time) # timing ends here for "start_time" variable
print(grid.best_estimator_)
print(grid.best_params_)


Fitting 4 folds for each of 16 candidates, totalling 64 fits
[CV 1/4] END .C=0.1, gamma=scale, kernel=linear;, score=0.557 total time=  35.1s
[CV 2/4] END .C=0.1, gamma=scale, kernel=linear;, score=0.567 total time=  35.0s
[CV 3/4] END .C=0.1, gamma=scale, kernel=linear;, score=0.542 total time=  34.8s
[CV 4/4] END .C=0.1, gamma=scale, kernel=linear;, score=0.562 total time=  35.8s
[CV 1/4] END ....C=0.1, gamma=scale, kernel=rbf;, score=0.385 total time=  36.0s
[CV 2/4] END ....C=0.1, gamma=scale, kernel=rbf;, score=0.414 total time=  36.3s
[CV 3/4] END ....C=0.1, gamma=scale, kernel=rbf;, score=0.440 total time=  36.3s
[CV 4/4] END ....C=0.1, gamma=scale, kernel=rbf;, score=0.435 total time=  36.9s
[CV 1/4] END ..C=0.1, gamma=auto, kernel=linear;, score=0.557 total time=  36.0s
[CV 2/4] END ..C=0.1, gamma=auto, kernel=linear;, score=0.567 total time=  36.0s
[CV 3/4] END ..C=0.1, gamma=auto, kernel=linear;, score=0.542 total time=  35.9s
[CV 4/4] END ..C=0.1, gamma=auto, kernel=linear;

# KNN

In [ ]:
 from sklearn.neighbors import KNeighborsClassifier

 neigh = KNeighborsClassifier(n_neighbors=5)

 neigh.fit(x_train, y_train)
 train_y_pred_neigh=neigh.predict(x_train)
 val_y_pred_neigh=neigh.predict(x_val)

 print(f'F1 score for the train {f1_score(y_train,  train_y_pred_neigh)}')
 print(f'F1 score for the val {f1_score(y_val, val_y_pred_neigh)}')



F1 score for the train 0.634770514603616
F1 score for the val 0.5026041666666666


'\nin K=3\nF1 score for the train 0.6966982339390837\nF1 score for the val 0.46782316731952994\n'

F1 score for the train 0.634770514603616

F1 score for the val 0.5026041666666666

In [ ]:
"""from sklearn.ensemble import RandomForestClassifier
random_forest_model = RandomForestClassifier(n_estimators=100,random_state=20)
random_forest_model.fit(x_train, y_train)
val_y_pred_tree=random_forest_model.predict(x_val)
print(f'F1 score for the val {f1_score(y_val, val_y_pred_tree)}')

"""

F1 score for the val 0.4069015097052479


# Model 2 Liner ANN

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

# We create a FC regression network, with 2 layers.
class RegressioNet(nn.Module):
   def __init__(self):
       super(RegressioNet, self).__init__()
       self.hidden_dim = 20
       self.layer_1 = torch.nn.Linear(200, self.hidden_dim)
       self.layer_2 = torch.nn.Linear(self.hidden_dim, 1)

   def forward(self, x):
       x = self.layer_1(x)        # x.size() -> [batch_size, self.hidden_dim]
       x = self.layer_2(x)        # x.size() -> [batch_size, 1]

       return x  

net = RegressioNet()
print(net)

RegressioNet(
  (layer_1): Linear(in_features=200, out_features=20, bias=True)
  (layer_2): Linear(in_features=20, out_features=1, bias=True)
)


In [25]:
# Define Optimizer and Loss Function

optimizer = torch.optim.Adam(net.parameters())
loss_func = torch.nn.CrossEntropyLoss()

In [26]:
batch_size = 20

# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(x_train).float(), torch.from_numpy(y_train).float())
valid_data = TensorDataset(torch.from_numpy(x_val).float(), torch.from_numpy(y_val).float())

#test_data = TensorDataset(torch.from_numpy(test_x).float(), torch.from_numpy(test_y).float())

# make sure the SHUFFLE your training data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=False, batch_size=batch_size)
#test_loader = DataLoader(test_data, shuffle=False, batch_size=batch_size)

In [27]:
# First checking if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if torch.cuda.is_available():
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')


Training on GPU.


In [28]:
# Define training params
epochs = 10

counter = 0
print_every = 100
clip = 1000 # gradient clipping

# move model to GPU, if available
net = net.float()
net.to(device)

net.train()
# train for some number of epochs


RegressioNet(
  (layer_1): Linear(in_features=200, out_features=20, bias=True)
  (layer_2): Linear(in_features=20, out_features=1, bias=True)
)

In [29]:
for e in range(epochs):
    # batch loop
    for inputs, labels in train_loader:
        counter += 1

        # if training on gpu
        inputs, labels = inputs.to(device), labels.to(device)

        # zero accumulated gradients
        net.zero_grad()

        # get the output from the model
        # x.size() -> [batch_size]
        batch_size = inputs.size(0)
        # IMPORTANT - change the dimensions of x before it enters the NN, batch size must always be first
        x = inputs.unsqueeze(0)         # x.size() -> [1, batch_size]
        x = x.view(batch_size, -1)      # x.size() -> [batch_size, 1]
        predictions = net(x)

        # calculate the loss and perform backprop
        loss = loss_func(predictions.squeeze(), labels.float())
        loss.backward()
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()

        # loss stats
        if counter % print_every == 0:
            # Get validation loss
            val_losses = []
            net.eval()
            print_flag = True
            for inputs, labels in valid_loader:
                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                if print_flag:
                  inputs, labels = zip(*sorted(zip(inputs.numpy(), labels.numpy())))
                  inputs = torch.from_numpy(np.asarray(inputs))
                  labels = torch.from_numpy(np.asarray(labels))
                inputs, labels = inputs.to(device), labels.to(device)


                # get the output from the model
                # x.size() -> [batch_size]
                batch_size = inputs.size(0)
                # IMPORTANT - change the dimensions of x before it enters the NN, batch size must always be first
                x = inputs.unsqueeze(0)    # x.size() -> [1, batch_size]
                x = x.view(batch_size, -1) # x.size() -> [batch_size, 1]
                val_predictions = net(x)
                val_loss = loss_func(val_predictions.squeeze(), labels.float())

                val_losses.append(val_loss.item())
                if print_flag:
                  print_flag = False
                  # plot and show learning process
                  fig = plt.figure()
                  ax = fig.add_subplot(111)
                  ax.cla()
                  ax.scatter(inputs.cpu().data.numpy(), labels.cpu().data.numpy())
                  ax.plot(inputs.cpu().data.numpy(), val_predictions.cpu().data.numpy(), 'r-', lw=2)
                  ax.text(0.5, 0, 'Loss=%.4f' % np.mean(val_losses), fontdict={'size': 10, 'color':  'red'})
                  plt.pause(0.1)
                  ax.clear()

            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))
plt.show()

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.